#$Imports$

In [ ]:
import os
import cv2 as cv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

#$Loading$ $Data$

In [ ]:
train_images = []
file_path = ["train/horses","train/humans"]
for path in file_path:
  for img in os.listdir(path):
    pic = cv.imread(os.path.join(path,img))
    pic = cv.resize(pic,(64,64),cv.BORDER_DEFAULT)
    pic = cv.cvtColor(pic,cv.COLOR_BGR2RGB)
    train_images.append(pic)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, #get all pixel values to 0-1, CNN converg faster on [0..1] data than on [0..255]
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory("train",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 126 images belonging to 3 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('validation',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 202 images belonging to 3 classes.


In [ ]:
class_names = ["Horse","Human"]

In [ ]:
def ShowImage(id,X_data):
  plt.imshow(X_data[id].reshape(64,64,3),cmap="gray")
  plt.axis("off")
  plt.show()

In [ ]:
id = np.random.randint(0,1027)
ShowImage(id,train_images)

IndexError: ignored

#$Basic$ $Models$

In [ ]:
cnn = tf.keras.models.Sequential()
# First layer
cnn.add(Conv2D(filters=64, kernel_size=5,strides=2,padding="same", activation='relu', input_shape=(64, 64, 3)))
cnn.add(MaxPool2D(pool_size=3, strides=2,padding="same"))
# Second layer
cnn.add(Conv2D(filters=128, kernel_size=3,strides=1,padding="same", activation='relu'))
cnn.add(MaxPool2D(pool_size=3, strides=2,padding="same"))
cnn.add(Flatten())
cnn.add(Dense(units=64, activation='relu'))
cnn.add((Dropout(0.7)))
cnn.add(Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])
classifier = cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
10/33 [========>.....................] - ETA: 40s - loss: 0.7309 - accuracy: 0.4969

In [ ]:
cnn.evaluate(test_set)

In [ ]:
for i in range(10):
    id = np.random.randint(0,1027)
    plt.imshow(train_images[id].reshape(64,64,3),cmap="gray")
    plt.axis("off")
    plt.title(f"Predicted : {class_names[int(cnn.predict(train_images[id].reshape(-1,64,64,3))[0][0])]}")
    plt.show()

#$Complex$ $Models$

In [ ]:
class ResNetBlock(Layer):
    def __init__(self,units,strides,**kwargs):
        super().__init__(**kwargs)
        self.filters = units
        self.stride = strides
    def build(self,batch_size):
        self.main_layers = [
            Conv2D(self.filters,3,self.stride,padding="same",use_bias=False,activation=None),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(self.filters,3,self.stride,padding="same",activation="relu"),
            BatchNormalization(),
        ]
        self.skip_layers = [
            Conv2D(self.filters,1,1,padding="same",activation="relu",kernel_initializer="he_normal"),
            BatchNormalization(),
        ]
        super().build(batch_size)
    def call(self,X):
        inputs = X
        skip_input = X
        for layers in self.main_layers:
            inputs = layers(inputs)
        for skip_layers in self.skip_layers:
            skip_input = skip_layers(skip_input)
        return Activation('relu')(inputs + skip_input)
    def get_config(self):
        return {"units":self.filters,"strides":self.stride}

In [ ]:
ResNetModel = keras.Sequential()
ResNetModel.add(InputLayer(input_shape=(64,64,3)))
ResNetModel.add(Conv2D(32,3,1,"same",activation="relu",kernel_initializer="he_normal"))
ResNetModel.add(MaxPool2D(3,2,"same"))
ResNetModel.add(ResNetBlock(64,1))
ResNetModel.add(ResNetBlock(64,1))
ResNetModel.add(MaxPool2D(3,2,"same"))
ResNetModel.add(Flatten())
ResNetModel.add(Dropout(0.7))
ResNetModel.add(Dense(1,activation="sigmoid"))

ResNetModel.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

ES_CB = keras.callbacks.EarlyStopping(patience=5)
ResNetModel.fit(
    training_set,
    validation_data=(test_set),
    epochs=10,
    callbacks=[ES_CB]
)

In [ ]:
ResNetModel.evaluate(test_set)


In [ ]:
for i in range(10):
    id = np.random.randint(0,1027)
    plt.imshow(train_images[id].reshape(64,64,3),cmap="gray")
    plt.axis("off")
    plt.title(f"Predicted : {class_names[int(ResNetModel.predict(train_images[id].reshape(-1,64,64,3))[0][0])]}")
    plt.show()

This is better than the normal CNN model we have.

In [ ]:
class ResNetBlock_02(Layer):
    def __init__(self,units,strides,**kwargs):
        super().__init__(**kwargs)
        self.filters = units
        self.stride = strides
    def build(self,batch_size):
        self.main_layers = [
            Conv2D(self.filters,3,self.stride,padding="same",use_bias=False,activation=None),
            BatchNormalization(),
            Activation('relu'),
            Conv2D(self.filters,3,self.stride,padding="same",activation="relu"),
            Dropout(0.3),
            BatchNormalization(),
        ]
        self.skip_layers = [
            Conv2D(self.filters,1,1,padding="same",activation="relu",kernel_initializer="he_normal"),
            Dropout(0.3),
            BatchNormalization(),
        ]
        super().build(batch_size)
    def call(self,X):
        inputs = X
        skip_input = X
        for layers in self.main_layers:
            inputs = layers(inputs)
        for skip_layers in self.skip_layers:
            skip_input = skip_layers(skip_input)
        return Activation('relu')(inputs + skip_input)
    def get_config(self):
        return {"units":self.filters,"strides":self.stride}

In [ ]:
ResNetModel_02 = keras.Sequential()
ResNetModel_02.add(InputLayer(input_shape=(64,64,3)))
ResNetModel_02.add(Conv2D(32,3,1,"same",activation="relu",kernel_initializer="he_normal"))
ResNetModel_02.add(MaxPool2D(3,2,"same"))
ResNetModel_02.add(ResNetBlock_02(64,1))
ResNetModel_02.add(ResNetBlock_02(64,1))
ResNetModel_02.add(MaxPool2D(3,2,"same"))
ResNetModel_02.add(Flatten())
ResNetModel_02.add(Dropout(0.7))
ResNetModel_02.add(Dense(1,activation="sigmoid"))

ResNetModel_02.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

ES_CB = keras.callbacks.EarlyStopping(patience=5)
ResNetModel_02.fit(
    training_set,
    validation_data=(test_set),
    epochs=5,
    callbacks=[ES_CB]
)

In [ ]:
ResNetModel_02.evaluate(test_set)

This is way better than the Previous ResNet Model.